In [1]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.4 MB/s eta 0:00:01


In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd 



[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/pranjalbhatt/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [4]:
## load dataset 
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

In [2]:
## data preprocessing 
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-12-01 13:51:41.718317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
## load the dataset 
with open('hamlet.txt','r') as f:
    text=f.read().lower()
    

In [14]:
# tokenize the text 
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1 # since it is starting from 0 index
print(total_words)

4818


In [29]:
## Creating input sequences 
## here we will create input and output sequ
input_sequences=[]
for line in text.split('\n'):
    tokenized_line=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(tokenized_line)):
        input_sequences.append(tokenized_line[:i+1])
# Example - 
# I am working in an IT firm - here for input sequence I am  working is output , beloware few more example 
# Input ~ Output 
# I am ~ working 
# I am working ~ in 
# I am working in ~ an 
# this is how we have prepared out input 

In [34]:
## Making sequences of same len 
# we will now take the max len of all the sequences and left pad each sequence with this max len 
max_seq_len=max([len(x) for x in input_sequences])
print(f"Max Sequence len in the corupus is {max_seq_len}") # there are max 14 words in a line 

input_sequences=pad_sequences(input_sequences,maxlen=max_seq_len,padding='pre')  # pre and post 2 options are possible 


Max Sequence len in the corupus is 14


In [36]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [39]:
# Dividing dataset into training and testing 
import tensorflow as tf
X,y=input_sequences[:,:-1],input_sequences[:,-1] # X is all word but last word and y is only last word
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [40]:
# converting target y to array 
# based on num of classes we have to convert y in vector format 
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [41]:
# splitting dataset to train and test 
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [ ]:
# Define Early stopping 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [45]:
# Train LSTM model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense,Dropout,LSTM

# Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_seq_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

2024-12-01 14:27:43.366702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-01 14:27:43.367750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-01 14:27:43.368846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [46]:
# compile the model 
loss=tf.keras.losses.categorical_crossentropy
model.compile(loss=loss,optimizer='adam',metrics=['accuracy'])

In [47]:
# training the model
history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),verbose=1,callbacks=[early_stopping])



Epoch 1/50


2024-12-01 14:27:46.728359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-01 14:27:46.729496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-01 14:27:46.730290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

602/604 [============================>.] - ETA: 0s - loss: 6.9314 - accuracy: 0.0319

2024-12-01 14:28:01.730151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-01 14:28:01.731058: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-01 14:28:01.731981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

604/604 [==============================] - 17s 24ms/step - loss: 6.9324 - accuracy: 0.0318 - val_loss: 6.7254 - val_accuracy: 0.0376
Epoch 2/50
604/604 [==============================] - 14s 22ms/step - loss: 6.4859 - accuracy: 0.0349 - val_loss: 6.8246 - val_accuracy: 0.0459
Epoch 3/50
604/604 [==============================] - 13s 22ms/step - loss: 6.3563 - accuracy: 0.0413 - val_loss: 6.8779 - val_accuracy: 0.0474
Epoch 4/50
604/604 [==============================] - 14s 23ms/step - loss: 6.2285 - accuracy: 0.0489 - val_loss: 6.9168 - val_accuracy: 0.0505
Epoch 5/50
604/604 [==============================] - 14s 23ms/step - loss: 6.0949 - accuracy: 0.0512 - val_loss: 6.8966 - val_accuracy: 0.0508
Epoch 6/50
604/604 [==============================] - 14s 23ms/step - loss: 5.9632 - accuracy: 0.0581 - val_loss: 6.9311 - val_accuracy: 0.0529
Epoch 7/50
604/604 [==============================] - 15s 24ms/step - loss: 5.8237 - accuracy: 0.0659 - val_loss: 6.9955 - val_accuracy: 0.0603
Epo

In [50]:
model.save('Saved Model/model.h5')

In [67]:
# Predicting the next word 
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_seq_len:
        token_list=token_list[-(max_seq_len-1):] # Ensure sequence len matches max Sequence 
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)[0]
    print(predicted_word_index)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [68]:
input_text="To be or not to be "
print(f"Input text :{input_text}")
# max_seq_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_seq_len)
print(f"Next word is {next_word}")

Input text :To be or not to be 
1659
Next word is blest


In [61]:
word_index_reverse={v:k for k,v in tokenizer.word_index.items()}
word_index_reverse[1659]

'blest'

In [66]:

predicted_word_index=1659
for word,index in tokenizer.word_index.items():
    if index==predicted_word_index:
        print(word)
    else:
        pass

blest


In [12]:
tokenizer.word_index


{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 